# Fiftyone

The open-source tool for building high-quality datasets and computer vision models.

<img src="https://voxel51.com/docs/fiftyone/_static/images/homepage_curate.gif" alt="drawing" width="500"/>

Install with pip in a python virtual environment (conda recommended).
```console
pip install fiftyone
```
Detailed instructions here: https://voxel51.com/docs/fiftyone/getting_started/install.html

In [1]:
import fiftyone as fo
fo.list_datasets()

[]

In [2]:
name = "tmp-dataset"
dataset_dir = "./datasets/"

if name in fo.list_datasets():
    fo.delete_dataset(name)

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.CVATVideoDataset,
    name=name,
)

# generate metadata for the dataset
dataset.compute_metadata()

# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

 100% |███████████████████| 28/28 [34.6s elapsed, 0s remaining, 0.6 samples/s]    
Computing metadata...
 100% |███████████████████| 28/28 [401.7ms elapsed, 0s remaining, 69.7 samples/s]      
Name:        tmp-dataset
Media type:  video
Num samples: 28
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.VideoMetadata)
Frame fields:
    id:           fiftyone.core.fields.ObjectIdField
    frame_number: fiftyone.core.fields.FrameNumberField
    detections:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
[<Sample: {
    'id': '63cd3ef6e09f3528fe9a4f04',
    'media_type': 'video',
    'filepath': '/Users/kevinserrano/GitHub/JKU_C0_CV/datasets/data/video_000.mp4',
    'tags': [],
    'metadata': <VideoMetadata: {

In [3]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


In [4]:
for sample in dataset:
    # add video name to the sample
    sample['filename'] = sample.filepath.split("/")[-1]
    sample.save()

In [5]:
for frame_number, frame in sample.frames.items():
    print(frame)
    break

<Frame: {
    'id': '63cd3f173260512d0ff8fa95',
    'frame_number': 1,
    'detections': <Detections: {
        'detections': [
            <Detection: {
                'id': '63cd3f16e09f3528fe9acbf1',
                'attributes': {},
                'tags': [],
                'label': 'person',
                'bounding_box': [0.25, 0.55859375, 0.04375, 0.05078125],
                'mask': None,
                'confidence': None,
                'index': 0,
                'keyframe': True,
            }>,
        ],
    }>,
}>


In [6]:
import random
import fiftyone.utils.random as four

# get sample IDs and shuffle them
sample_ids = dataset.values("id")
random.seed(51)
random.shuffle(sample_ids)

# split the dataset into 5 folds
folds = [sample_ids[i::5] for i in range(5)]

# tag each sample with the fold number
for i, fold in enumerate(folds):
    dataset.select(fold).tag_samples(f"fold_{i+1}")
    print(f"Fold {i+1} has {len(fold)} samples")
    print(dataset.select(fold).values("filename"))
        

Fold 1 has 6 samples
['video_004.mp4', 'video_010.mp4', 'video_013.mp4', 'video_016.mp4', 'video_018.mp4', 'video_024.mp4']
Fold 2 has 6 samples
['video_003.mp4', 'video_006.mp4', 'video_014.mp4', 'video_019.mp4', 'video_022.mp4', 'video_026.mp4']
Fold 3 has 6 samples
['video_001.mp4', 'video_007.mp4', 'video_020.mp4', 'video_021.mp4', 'video_023.mp4', 'video_027.mp4']
Fold 4 has 5 samples
['video_000.mp4', 'video_005.mp4', 'video_012.mp4', 'video_015.mp4', 'video_025.mp4']
Fold 5 has 5 samples
['video_002.mp4', 'video_008.mp4', 'video_009.mp4', 'video_011.mp4', 'video_017.mp4']


In [7]:
# convert video to frames
import fiftyone.core.video as fcv
frames_dataset = fcv.make_frames_dataset(
    dataset,
    sample_frames=True,
    fps=3,
)

Setting 3502 frame filepaths on the input collection that exist on disk but are not recorded on the dataset


In [8]:
session.view = frames_dataset.view()

In [10]:
classes = frames_dataset.distinct("detections.detections.label")

# export datasets for cross validation
for i in range(len(folds)):
    export_dir = f"./datasets/CV-C0-v1-k5-f{i+1}"

    train_split = frames_dataset.match_tags(f"fold_{i+1}", bool=False)
    val_split = frames_dataset.match_tags(f"fold_{i+1}", bool=True)

    for split in zip(["train", "val"], [train_split, val_split]):
        split_name, view = split
        view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            split=split_name,
            label_field="detections",
            classes=classes,
        )
    

Directory './datasets/CV-C0-v1-k5-f1' already exists; export will be merged with existing files
 100% |███████████████| 2663/2663 [9.2s elapsed, 0s remaining, 284.1 samples/s]       
Directory './datasets/CV-C0-v1-k5-f1' already exists; export will be merged with existing files
 100% |█████████████████| 839/839 [2.8s elapsed, 0s remaining, 272.3 samples/s]      
Directory './datasets/CV-C0-v1-k5-f2' already exists; export will be merged with existing files
 100% |███████████████| 2645/2645 [7.5s elapsed, 0s remaining, 422.6 samples/s]       
Directory './datasets/CV-C0-v1-k5-f2' already exists; export will be merged with existing files
 100% |█████████████████| 857/857 [2.4s elapsed, 0s remaining, 365.0 samples/s]      
Directory './datasets/CV-C0-v1-k5-f3' already exists; export will be merged with existing files
 100% |███████████████| 2593/2593 [5.0s elapsed, 0s remaining, 722.1 samples/s]      
Directory './datasets/CV-C0-v1-k5-f3' already exists; export will be merged with existin